### Importing the required libraries

And import the csv file

In [1]:
import folium
import pandas as pd
data = pd.read_csv('volcano_data.csv')
data.head()

,Year,Month,Day,TSU,EQ,Name,Location,Country,Latitude,Longitude,...,TOTAL_DEATHS,TOTAL_DEATHS_DESCRIPTION,TOTAL_MISSING,TOTAL_MISSING_DESCRIPTION,TOTAL_INJURIES,TOTAL_INJURIES_DESCRIPTION,TOTAL_DAMAGE_MILLIONS_DOLLARS,TOTAL_DAMAGE_DESCRIPTION,TOTAL_HOUSES_DESTROYED,TOTAL_HOUSES_DESTROYED_DESCRIPTION
0,2010,1,NaN,NaN,NaN,Tungurahua,Ecuador,Ecuador,-1.467,-78.442,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
1,2010,3,31.0,NaN,NaN,Eyjafjallajokull,Iceland-S,Iceland,63.630,-19.620,...,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010,5,27.0,NaN,NaN,Pacaya,Guatemala,Guatemala,14.381,-90.601,...,1.0,1.0,3.0,1.0,NaN,NaN,NaN,1.0,3.0,1.0
3,2010,5,29.0,TSU,EQ,Sarigan,Mariana Is-C Pacific,United States,16.708,145.780,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010,8,6.0,NaN,NaN,Karangetang [Api Siau],Sangihe Is-Indonesia,Indonesia,2.780,125.480,...,4.0,1.0,NaN,NaN,5.0,1.0,NaN,NaN,NaN,1.0


In [2]:
data.columns

Index(['Year', 'Month', 'Day', 'TSU', 'EQ', 'Name', 'Location', 'Country',
       'Latitude', 'Longitude', 'Elevation', 'Type', 'Status', 'Time', 'VEI',
       'Agent', 'DEATHS', 'DEATHS_DESCRIPTION', 'MISSING',
       'MISSING_DESCRIPTION', 'INJURIES', 'INJURIES_DESCRIPTION',
       'DAMAGE_MILLIONS_DOLLARS', 'DAMAGE_DESCRIPTION', 'HOUSES_DESTROYED',
       'HOUSES_DESTROYED_DESCRIPTION', 'TOTAL_DEATHS',
       'TOTAL_DEATHS_DESCRIPTION', 'TOTAL_MISSING',
       'TOTAL_MISSING_DESCRIPTION', 'TOTAL_INJURIES',
       'TOTAL_INJURIES_DESCRIPTION', 'TOTAL_DAMAGE_MILLIONS_DOLLARS',
       'TOTAL_DAMAGE_DESCRIPTION', 'TOTAL_HOUSES_DESTROYED',
       'TOTAL_HOUSES_DESTROYED_DESCRIPTION'],
      dtype='object')

Subset the data to get only the rows and columns we want

In [3]:
data_new = data.loc[:,("Year","Name","Country","Latitude","Longitude","Type","Elevation","TOTAL_DEATHS")]
data_new.head()

,Year,Name,Country,Latitude,Longitude,Type,Elevation,TOTAL_DEATHS
0,2010,Tungurahua,Ecuador,-1.467,-78.442,Stratovolcano,5023,NaN
1,2010,Eyjafjallajokull,Iceland,63.630,-19.620,Stratovolcano,1666,2.0
2,2010,Pacaya,Guatemala,14.381,-90.601,Complex volcano,2552,1.0
3,2010,Sarigan,United States,16.708,145.780,Stratovolcano,538,NaN
4,2010,Karangetang [Api Siau],Indonesia,2.780,125.480,Stratovolcano,1784,4.0


In [4]:
##Always check if any null oresent or not
data_new.isnull().sum()

Year             0
Name             0
Country          0
Latitude         0
Longitude        0
Type             0
Elevation        0
TOTAL_DEATHS    34
dtype: int64

In [5]:
data_new = data_new.fillna(0)
data_new.isnull().sum()

Year            0
Name            0
Country         0
Latitude        0
Longitude       0
Type            0
Elevation       0
TOTAL_DEATHS    0
dtype: int64

## Formating the map

In [6]:
map = folium.Map(location = [40.52,34.34], zoom_start = 2,tiles = "Stamen Terrain",)
##add marker
fg = folium.FeatureGroup(name = "My Map")

In [7]:
data_new.columns


Index(['Year', 'Name', 'Country', 'Latitude', 'Longitude', 'Type', 'Elevation',
       'TOTAL_DEATHS'],
      dtype='object')

In [8]:
#subset desired properties to display in popup
year_Eruppted = list(data_new["Year"])
name_Volcano = list(data_new["Name"])
country_Volcano = list(data_new["Country"])
latitude_Volcano = list(data_new["Latitude"])
longitude_Volcano = list(data_new["Longitude"])
type_Volcano = list(data_new["Type"])
elevation_Volcano = list(data_new["Elevation"])
death = list(data_new["TOTAL_DEATHS"])


In [9]:
html = """
Volcano name:
<a href="https://www.google.com/search?q=%%22%s%%22" target="_blank">%s</a><br>
Last Eruption Year: %s <br>
Elevation: %s m <br>
Country: %s <br>
Cordinates: [ %s , %s]<br>
Type: %s
Total Deaths = %s

"""
for typev,lt,ln,el,namev,year,country,death in zip(type_Volcano,latitude_Volcano,longitude_Volcano,elevation_Volcano,name_Volcano,year_Eruppted,country_Volcano,death):
    if el<1000:
        type_color = "green"
    elif 1000<=el<3000:
        type_color = "orange"
    else:
        type_color = "red"
    iframe = folium.IFrame(html=html % (namev,namev,year,el,country,lt,ln,typev,death), width=200, height=120)
    fg.add_child(folium.Marker(location = [lt, ln],
    #popup =folium.Popup(iframe,"Year:"+str(year)+'<br>'+"Elevation: " + str(el) +"m"+ '<br>'+"Name: " + str(namev)+'<br>'+'Country:'+str(country)+'<br>'+"Coordinates:{"+str(lt)+","+str(ln)+"]"+'<br>'+"Type:" + str(typev)),icon = folium.Icon(color = type_color),tooltip =folium.Tooltip("Volcano "+namev,style=None, sticky=True)))
    popup =folium.Popup(iframe),icon = folium.Icon(color = type_color),tooltip =folium.Tooltip("Volcano "+namev,style=None, sticky=True)))


In [10]:
fgv = folium.FeatureGroup(name = "Population")
data_json = open("world.json", 'r', encoding='utf-8-sig').read()

fgv.add_child(folium.GeoJson(data=data_json ,
style_function=lambda x: {'fillColor':'green' if x['properties']['POP2005'] <= 10000000
else 'orange' if 10000000 < x['properties']['POP2005'] < 20000000 else 'red'}))
loc = 'Volcano map with population layer'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc)
map.get_root().html.add_child(folium.Element(title_html))
map.add_child(fg)
map.add_child(fgv)
map.add_child(folium.LayerControl())
map.save("Volcano_new.html")